In [1]:
import math
from itertools import combinations
from collections import namedtuple
from ortools.sat.python import cp_model
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2

In [2]:
import sys
file_location = "C:/Users/VZ/Documents/vrp/data/vrp_200_16_2"
with open(file_location, 'r') as input_data_file:
    input_data = input_data_file.read()

In [ ]:
# Здесь должен быть код формирования матрицы расстояний dis, вектора спроса, вектора вместимости и переменной количества машин

In [4]:
def create_data_model(dis,vehicle_count,demands,v_cap):
    data = {}
    data['distance_matrix'] = dis
    data['demands'] = demands
    data['vehicle_capacities'] = v_cap
    data['num_vehicles'] = vehicle_count
    data['depot'] = 0
    return data

def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    out=''
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            out+='{} '.format(node_index)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        out+='{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))
    output_data=str(total_distance)+' 0\n'
    output_data=output_data+out
    return output_data

In [5]:
# Instantiate the data problem.
data = create_data_model(dis,vehicle_count,demands,v_cap)

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)

In [6]:
# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


# Add Capacity constraint.
def demand_callback(from_index):
    """Returns the demand of the node."""
    # Convert from routing variable Index to demands NodeIndex.
    from_node = manager.IndexToNode(from_index)
    return data['demands'][from_node]

demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(10)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    output_data=print_solution(data, manager, routing, solution)

print(output_data)

Objective: 16425
Route for vehicle 0:
 0 Load(0.0) ->  72 Load(16.0) ->  31 Load(41.0) ->  119 Load(50.0) ->  164 Load(70.0) ->  84 Load(87.0) ->  14 Load(118.0) ->  133 Load(132.0) ->  19 Load(147.0) ->  83 Load(173.0) ->  154 Load(175.0) ->  166 Load(200.0) ->  0 Load(200.0)
Distance of the route: 2103m
Load of the route: 200.0

Route for vehicle 1:
 0 Load(0.0) ->  199 Load(17.0) ->  113 Load(37.0) ->  91 Load(50.0) ->  22 Load(62.0) ->  114 Load(70.0) ->  122 Load(99.0) ->  36 Load(111.0) ->  138 Load(142.0) ->  103 Load(161.0) ->  5 Load(182.0) ->  128 Load(191.0) ->  70 Load(199.0) ->  0 Load(199.0)
Distance of the route: 1524m
Load of the route: 199.0

Route for vehicle 2:
 0 Load(0.0) ->  118 Load(17.0) ->  56 Load(36.0) ->  25 Load(50.0) ->  148 Load(80.0) ->  92 Load(91.0) ->  135 Load(96.0) ->  145 Load(97.0) ->  73 Load(107.0) ->  146 Load(134.0) ->  18 Load(147.0) ->  147 Load(183.0) ->  181 Load(199.0) ->  0 Load(199.0)
Distance of the route: 1158m
Load of the route: 199.